<a href="https://colab.research.google.com/github/muhbagasaryad/Machine-Learning/blob/main/3_Model_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [146]:
import numpy as np 
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import metrics

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
df = pd.read_csv('')

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.shape

### **Preprocessing Data**

In [ ]:
#change object to float64
df['Intensitas Curah Hujan (mm)'] = pd.to_numeric(df['Intensitas Curah Hujan (mm)'],errors = 'coerce')
df.info()

In [ ]:
df

In [ ]:
df['Kategori Banjir'].unique()

In [ ]:
df.isnull().sum()

In [ ]:
df.fillna((0), inplace = True)
df.head()

In [ ]:
df.isnull().sum()

In [ ]:
banjir_df=df.drop(columns='Kategori Banjir')
banjir_df.head()

In [ ]:
banjir_df.describe()

In [ ]:
banjir_df.corr()

In [ ]:
plt.figure(figsize=(8, 8))
sns.heatmap(banjir_df.corr(),annot=True)

In [ ]:
banjir_df.hist(figsize=(8,8))
plt.show()

In [ ]:
sns.scatterplot(x='Tinggi Muka Air (m)', y='Intensitas Curah Hujan (mm)', data=banjir_df)
plt.tight_layout()

In [ ]:
df['Kategori Banjir'].replace(['aman','Siaga1','Siaga2'],[0,1,2],inplace=True)
df.head()

In [ ]:
df['Kategori Banjir'].value_counts()

In [ ]:
#memisahkan data untuk prediksi
x = df.iloc[:,1:3]
x.head()

In [ ]:
#memisahkan kolom kategori dari dataset
y = df.iloc[:, -1]
y.head()

### **Decision Tree**

In [ ]:
import time

start = time.time()
from sklearn import model_selection
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,recall_score,confusion_matrix,plot_confusion_matrix
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.4,random_state=0)
print("Training dataset: ", x_train.shape[0])
print("Test dataset: ", x_test.shape[0])

In [ ]:
#data train
x_train.head()

In [ ]:
y_train.head()

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
dt_clf = DecisionTreeClassifier(max_depth=3, random_state=0)
dt_clf.fit(x_train,y_train)
dt_clf_acc = cross_val_score(dt_clf,x_train,y_train,cv=3,scoring="accuracy",n_jobs=-1)
dt_clf_acc

In [ ]:
#Training Data
y_pred = dt_clf.predict(x_train)
print("\naccuracy score:%f"%(accuracy_score(y_train,y_pred)*100))

In [ ]:
print('Confusion Matrix: \n')
print(plot_confusion_matrix(dt_clf, x_train, y_train))
print(metrics.classification_report(y_train, y_pred))

In [ ]:
stop = time.time()
print(f"Training time: {stop - start}s")

In [174]:
#Predicted flood chances
start1 = time.time()

y_pred = dt_clf.predict(x_test)
#print(y_pred)

In [175]:
#Actual flood chances
#print("actual values:")
#print(y_test.values)

In [ ]:
#Test Data
print("\naccuracy score:%f"%(accuracy_score(y_test,y_pred)*100))

In [ ]:
#cm=confusion_matrix(y_test,y_pred)
print('Confusion Matrix: \n')
print(plot_confusion_matrix(dt_clf, x_test, y_test))
print(metrics.classification_report(y_test, y_pred))

In [ ]:
stop1 = time.time()
print(f"Testing time: {stop1 - start1}s")

In [ ]:
# Visualize the tree by graphiz
import graphviz
from sklearn import tree
feature_names = ['Height (m)','Intensitas Hujan (mm)']
dot_data = tree.export_graphviz(dt_clf, out_file=None, filled=True, rounded = True, feature_names=feature_names)
graph = graphviz.Source(dot_data)
graph

In [ ]:
x_new=[[5,40]]
pred = dt_clf.predict(x_new)
pred

In [ ]:
import pickle
pickle.dump(dt_clf, open('modeldt.pkl', 'wb'))
model = pickle.load(open('modeldt.pkl', 'rb'))
print(model)

### **Random Forest**

In [ ]:
import time

start2 = time.time()
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,recall_score,confusion_matrix,plot_confusion_matrix
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.4)
rf = RandomForestClassifier(n_estimators=100,max_depth=3,random_state=0)
rf_clf = rf.fit(x_train,y_train)
rf_clf

In [ ]:
print("Training dataset: ", x_train.shape[0])
print("Test dataset: ", x_test.shape[0])

In [ ]:
from sklearn.model_selection import cross_val_score
rf_clf_acc = cross_val_score(rf_clf,x_train,y_train,cv=3,scoring="accuracy",n_jobs=-1)
rf_clf_acc

In [185]:
y_pred = rf_clf.predict(x_train)

In [ ]:
#Training data
print("\naccuracy score:%f"%(accuracy_score(y_train,y_pred)*100))

In [ ]:
print('Confusion Matrix: \n')
print(plot_confusion_matrix(rf_clf, x_train, y_train))
print(metrics.classification_report(y_train, y_pred))

In [ ]:
stop2 = time.time()
print(f"Training time: {stop2 - start2}s")

In [189]:
start3 = time.time()

y_pred = rf_clf.predict(x_test)

In [ ]:
#Testing data
print("\naccuracy score:%f"%(accuracy_score(y_test,y_pred)*100))

In [ ]:
#cm=confusion_matrix(y_test,y_pred)
print('Confusion Matrix: \n')
print(plot_confusion_matrix(rf_clf, x_test, y_test))
print(metrics.classification_report(y_test, y_pred))

In [ ]:
stop3 = time.time()
print(f"Testing time: {stop3 - start3}s")

In [ ]:
x_new=[[4,70]]
pred = rf_clf.predict(x_new)
pred

In [ ]:
import pickle
pickle.dump(rf_clf, open('modelrf.pkl', 'wb'))
model = pickle.load(open('modelrf.pkl', 'rb'))
print(model)

### **Naive Bayes**

In [ ]:
import time

start4 = time.time()
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,recall_score,roc_auc_score,confusion_matrix, plot_confusion_matrix
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size = 0.4,random_state = 0)
print("Training dataset: ", x_train.shape[0])
print("Test dataset: ", x_test.shape[0])

In [196]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import GaussianNB
count_vector = CountVectorizer()
training_data = count_vector.fit_transform(x_train)
testing_data = count_vector.transform(x_test)
naive_bayes = GaussianNB()

In [197]:
naive_bayes.fit(x_train,y_train)
y_pred = naive_bayes.predict(x_train)

In [ ]:
#Training data
print("\naccuracy score:%f"%(accuracy_score(y_train,y_pred)*100))

In [ ]:
print('Confusion Matrix: \n')
print(plot_confusion_matrix(naive_bayes, x_train, y_train))
print(metrics.classification_report(y_train, y_pred))

In [ ]:
stop4 = time.time()
print(f"Training time: {stop4 - start4}s")

In [201]:
start5 = time.time()

naive_bayes.fit(x_train,y_train)
y_pred = naive_bayes.predict(x_test)

In [ ]:
#Testing data
print("\naccuracy score:%f"%(accuracy_score(y_test,y_pred)*100))

In [ ]:
#cm=confusion_matrix(y_test,y_pred)
print('Confusion Matrix: \n')
print(plot_confusion_matrix(naive_bayes, x_test, y_test))
print(metrics.classification_report(y_test, y_pred))

In [ ]:
stop5 = time.time()
print(f"Testing time: {stop5 - start5}s")

In [ ]:
x_new=[[7,40]]
pred = naive_bayes.predict(x_new)
pred

In [ ]:
import pickle
pickle.dump(naive_bayes, open('modelgnb.pkl', 'wb'))
model = pickle.load(open('modelgnb.pkl', 'rb'))
print(model)

### **Visualisasi Data**

In [ ]:
sns.scatterplot(x='Tinggi Muka Air (m)', y='Intensitas Curah Hujan (mm)', data=df, hue='Kategori Banjir')
plt.tight_layout()

### **Comparing 3 Model ML**

In [ ]:
models = []
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
models.append(('DT', DecisionTreeClassifier()))
models.append(('RF', RandomForestClassifier()))
models.append(('NB', GaussianNB()))


names = []
scores = []
for name, model in models:
    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)
    scores.append(accuracy_score(y_test, y_pred)*100)
    names.append(name)
tr_split = pd.DataFrame({'Name': names, 'Score': scores})
print(tr_split)

In [ ]:
import seaborn as sns
axis = sns.barplot(x = 'Name', y = 'Score', data =tr_split )
axis.set(xlabel='Classifier', ylabel='Accuracy')
for p in axis.patches:
    height = p.get_height()
    axis.text(p.get_x() + p.get_width()/2, height + 0.005, '{:1.4f}'.format(height), ha="center") 
    
plt.show()